In [34]:
import pandas as pd
import numpy as np
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

# Load data
orders = pd.read_csv('orders.xls')


# Create user-item matrix
user_item_matrix = orders.pivot_table(index='client', columns='product', values='quantity', fill_value=0)

# Load data into Surprise format
reader = Reader(rating_scale=(0, 100))
data = Dataset.load_from_df(orders[['client', 'product', 'quantity']], reader)

# Build SVD algorithm
algo = SVD()

# Evaluate algorithm performance
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Fit the algorithm on the whole dataset
trainset = data.build_full_trainset()
algo.fit(trainset)

# Get list of all product IDs and names
products_dict = dict(zip(orders['product'], orders['product_description']))

# Get list of all product IDs
product_ids = orders['product'].unique()

# Generate recommended products for customer 1001
recommended_products = []
for product_id in product_ids:
    predicted_rating = algo.predict('1128254', product_id).est
    recommended_products.append((product_id, predicted_rating))

# Sort recommended products by predicted rating
recommended_products.sort(key=lambda x: x[1], reverse=True)

# Print top 5 recommended products with names
for i, (product_id, predicted_rating) in enumerate(recommended_products[:50]):
    product_name = products_dict.get(product_id, 'Unknown')
    print(f'{i+1}. {product_name} (ID: {product_id}, predicted rating: {predicted_rating:.2f})')


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    614.51731134.11151298.4008377.8837912.9869867.5800335.4253
MAE (testset)     108.1693110.9050109.5647104.7487108.8808108.45372.0608  
Fit time          2.53    2.52    2.64    2.51    2.58    2.56    0.05    
Test time         0.83    0.37    0.62    0.37    0.71    0.58    0.18    
1. AGUA DAS PEDRAS SALGADAS 6x0.33 PET (ID: 126898, predicted rating: 100.00)
2. COCA COLA LATA 0.33 (ID: 558440, predicted rating: 100.00)
3. COMPAL CLASSICO PESSEGO 0,20 GFA (15) (ID: 202791, predicted rating: 100.00)
4. COMPAL CLASSICO PERA 0,20 GFA (15) (ID: 202790, predicted rating: 100.00)
5. ADOCANTE PO STICKS CX.250 RIO BRAVO (ID: 124867, predicted rating: 100.00)
6. ANANAS EM CALDA LATA PL 820G TREVI (ID: 126438, predicted rating: 100.00)
7. AZEITE REF+VIRG OLIVEIR SERRA GFAO 3 LT (ID: 565603, predicted rating: 100.00)
8. VINHO BRANCO VIRTUOSO PCT LT CX1